In [ ]:
import tensorflow as tf

hello_constant = tf.constant('Hello World')

with tf.Session() as sess:
    output = sess.run(hello_constant)
    print(output)

In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.int32)
with tf.Session() as sess:
    output = sess.run(x,feed_dict={x:123})
    print(output)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def get_weights(n_features,n_labels):
    return tf.Variable(tf.truncated_normal((n_features,n_labels)))

def get_bias(n_labels):
    return tf.Variable(tf.zeros(n_labels))

def linear(input,weights,bias):
    return tf.add(tf.matmul(input,weights),bias)

def softmax(logits):
    return tf.nn.softmax(logits)

def cross_entropy(softmax_data,onehot_data):
    return -tf.reduce_sum(tf.multiply(onehot_data,tf.log(softmax_data)))

In [ ]:
# one hot encoding
import numpy as np
from sklearn import preprocessing

labels = np.array([1,3,5,2,5,4,2,1])

# create the encoder
lb = preprocessing.LabelBinarizer()

# encoder assigns one hot vectors identifying the classes
lb.fit(labels)

#now transform the labels to one hot encoded vectors
lb.transform(labels)

In [ ]:
# End to End Example - Multi Layer perceptron

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".",one_hot=True,reshape=False)

In [ ]:
#Hyper Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128
display_step = 1

In [ ]:
# parameters
n_input = 784 # MNIST data shape (28*28)
n_classes = 10 # MNIST total classes (0 - 9 digits)
n_hidden_layer = 256 

#store weights and bias based on these features
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input,n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer,n_classes]))
}

biases = {
    'hidden_layer':tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

x = tf.placeholder("float",[None,28,28,1])
y = tf.placeholder("float",[None,n_classes])

x_flat = tf.reshape(x,[-1,n_input])

In [ ]:
# LAYER 1

layer_1 = tf.add(tf.matmul(x_flat,weights['hidden_layer']),biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)

In [ ]:
logits = tf.add(tf.matmul(layer_1,weights['out']),biases['out'])

In [ ]:
# COST
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
init = tf.global_variables_initializer()

In [ ]:


with tf.Session() as session:
    session.run(init)
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            session.run(optimizer,feed_dict={x:batch_x,y:batch_y})
            
        if epoch%display_step == 0:
            c = session.run(cost,feed_dict={x:batch_x,y:batch_y})
            print("Epoch:",'%04d' % (epoch+1),"cost =","{:.9f}".format(c))
            
    print("Optimization Finished!")        
    
    # Testing Model
    correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

    test_size = 256
    print("Accuracy:",accuracy.eval({x:mnist.test.images[:test_size],y:mnist.test.labels[:test_size]}))